In [1]:
# For modules loading
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer

# For BM25
import src.bm25_IR

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the pretrained T5 model and tokenizer
model_name = "strombergnlp/dant5-large"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(DEVICE)


In [ ]:
#for question, documents in tqdm(zip(dev_set['question, str'], dev_set['tf_idf_k1']), desc='Answering questions'):

# Example question and context
question = dev_set.loc[0, 'question, str']
documents = sparse_retrieval(question, X, k=3)

# Format the input for T5
input_text = f"Relevante paragraffer: {documents}\nSpørgsmål: {question}\nIndsæt svar her baseret på de relevante paragraffer:"

# Tokenize the input and generate an answer
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)

max_length = len(input_ids[0]) + 100

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        # generation set to stop at '.' as it otherwise just repeats itself (think it's because we don't sample)
        eos_token_id=tokenizer.encode(' Spørgsmål')[0]
    )

# Decode and print the generated answer
answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')
print(input_text, '\n')
print("Svar:", answer)